In [2]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import os
import gc
import datetime
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, mean_squared_error, precision_score

In [3]:
maindir = "/home/anerdi/Desktop/Zillow"

In [4]:
# linux
logerror = pd.read_csv(maindir + "/data/train_2016_v2.csv/train_2016_v2.csv")
logerror['weeknumber'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])
logerror['month'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month)
properties = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv")
test_parcels = pd.read_csv(maindir + "/data/sample_submission.csv", usecols = ['ParcelId'])
test_parcels.rename(columns={'ParcelId':'parcelid'}, inplace=True)

# join on parcel id
data = pd.merge(properties,logerror[['parcelid','logerror','month']], on='parcelid')

/home/anerdi/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Simple Data Preprocessing

In [5]:
from sklearn.preprocessing import LabelEncoder, Imputer 

In [6]:
data.columns

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildin

In [20]:
# Setup variables considered in the model

# numerical variables
num_atts = ['bathroomcnt','bedroomcnt','buildingqualitytypeid','calculatedbathnbr','finishedfloor1squarefeet',
           'calculatedfinishedsquarefeet','finishedsquarefeet12','finishedsquarefeet13',
           'finishedsquarefeet15','finishedsquarefeet50','finishedsquarefeet6','fireplacecnt',
           'fullbathcnt','garagecarcnt','garagetotalsqft','latitude','longitude','lotsizesquarefeet',
           'poolcnt','poolsizesum','censustractandblock','roomcnt','threequarterbathnbr','unitcnt',
           'yardbuildingsqft17','yardbuildingsqft26','numberofstories',
            'structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt','taxamount']

# categorical variables
cat_atts = ['airconditioningtypeid','architecturalstyletypeid',
           'buildingclasstypeid','heatingorsystemtypeid','pooltypeid10','pooltypeid2',
            'pooltypeid7','propertylandusetypeid','rawcensustractandblock','regionidcounty','regionidcity',
            'regionidzip','regionidneighborhood',
           'storytypeid','typeconstructiontypeid','yearbuilt','fireplaceflag',
           'taxdelinquencyflag']

In [21]:
def preprocess(data):
    X = data[num_atts + cat_atts].copy()
    # fill in missing values
    for c in num_atts:
        X[c].fillna(X[c].mean(skipna=True), inplace=True)
        X[c] = X[c].astype('float32')

    for c in cat_atts:
        if X[c].dtype == object:
    #         print(c)
            X[c].fillna("-1", inplace=True)
            X[c] = X[c].astype(str)
        else:
            X[c].fillna(-1, inplace=True)
        lbl = LabelEncoder()
        lbl.fit(list(X[c].values))
        X[c] = lbl.transform(X[c])
        X[c] = X[c].astype('float32')
    return X

## Submission functions

In [22]:
# def generate_submissions(oct_model,nov_model,dec_model,name='new_submission'):
#     """
#     This function creates the submission file for the public leaderboard predictions.
#     Three already fitted models, one for each of the predicting time points, is required.
#     """
#     models = {'201610': oct_model, '201611': nov_model, '201612': dec_model}
#     submission_df = DataFrame()
#     for i in range(int(properties.shape[0] / 100000)):
#         subset = properties.iloc[i*100000:(i+1)*100000].copy()
#         predictions = {}
#         for mm in range(10,13):
#             subset['month'] = mm
#             all_feats = preprocess(subset)
#             predictions['2016{x}'.format(x=mm)] = models['2016{x}'.format(x=mm)].predict(all_feats)
#         foo = subset[['parcelid']].reset_index(drop=True)
#         foo = pd.concat([foo, DataFrame(predictions)], axis=1)
# #         foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
# #                                                         '201611': nov_model.predict(all_feats),
# #                                                         '201612': dec_model.predict(all_feats)})], axis=1)
#         submission_df = pd.concat([submission_df, foo], ignore_index=True)

#     #  fencepost problem
#     subset = properties.iloc[2900000:].copy()
#     predictions = {}
#     for mm in range(10,13):
#         subset['month'] = mm
#         all_feats = preprocess(subset)
#         predictions['2016{x}'.format(x=mm)] = models['2016{x}'.format(x=mm)].predict(all_feats)
#     foo = subset[['parcelid']].reset_index(drop=True)
#     foo = pd.concat([foo, DataFrame(predictions)], axis=1)
# #     foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
# #                                                     '201611': nov_model.predict(all_feats),
# #                                                     '201612': dec_model.predict(all_feats)})], axis=1)
#     submission_df = pd.concat([submission_df, foo], ignore_index=True)
    
#     submission_df['201710'] = 0
#     submission_df['201711'] = 0
#     submission_df['201712'] = 0
    
#     submission_df.rename(columns={'parcelid':'ParcelId'}, inplace=True)    
#     submission_df[['201610','201611','201612','201710','201711','201712']]= submission_df[['201610','201611','201612',
#                                                                                            '201710','201711','201712']].round(4)
#     # unit test
#     submission_df.drop_duplicates(inplace=True)
#     assert submission_df.shape[0] == properties.shape[0]
#     # write to .csv
#     submission_df[['ParcelId','201610','201611','201612',
#                   '201710','201711','201712']].to_csv(name + ".csv", index=False)
#     return submission_df

In [23]:
def generate_submissions(oct_model,nov_model,dec_model,name='new_submission',logy=True):
    """
    This function creates the submission file for the public leaderboard predictions.
    Three already fitted models, one for each of the predicting time points, is required.
    """
    submission_df = DataFrame()
    for i in range(int(properties.shape[0] / 100000)):
        all_feats = preprocess(properties.iloc[i*100000:(i+1)*100000])
        foo = properties.iloc[i*100000:(i+1)*100000][['parcelid']].reset_index(drop=True)
        if logy:
            foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                            '201611': nov_model.predict(all_feats),
                                                            '201612': dec_model.predict(all_feats)})], axis=1)
        else:
            foo = pd.concat([foo, DataFrame({'201610': np.log(oct_model.predict(all_feats)),
                                                            '201611': np.log(nov_model.predict(all_feats)),
                                                            '201612': np.log(dec_model.predict(all_feats))})], axis=1)
        submission_df = pd.concat([submission_df, foo], ignore_index=True)

    #  fencepost problem
    all_feats = preprocess(properties.iloc[2900000:])
    foo = properties.iloc[2900000:][['parcelid']].reset_index(drop=True)
    foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                    '201611': nov_model.predict(all_feats),
                                                    '201612': dec_model.predict(all_feats)})], axis=1)
    submission_df = pd.concat([submission_df, foo], ignore_index=True)
    
    submission_df['201710'] = 0
    submission_df['201711'] = 0
    submission_df['201712'] = 0
    
    submission_df.rename(columns={'parcelid':'ParcelId'}, inplace=True)    
    submission_df[['201610','201611','201612','201710','201711','201712']]= submission_df[['201610','201611','201612',
                                                                                           '201710','201711','201712']].round(4)
    # unit test
    submission_df.drop_duplicates(inplace=True)
    assert submission_df.shape[0] == properties.shape[0]
    # write to .csv
    submission_df[['ParcelId','201610','201611','201612',
                  '201710','201711','201712']].to_csv(name + ".csv", index=False)
    return submission_df

In [24]:
def mean_absolute_errors(submission_df, comparison_df):
    """
    This function takes a submission entry for public leaderboard, and returns
    the training error for each month.
    """
    # training error
    trainresults = pd.merge(submission_df[['ParcelId','201610','201611','201612']], comparison_df[['parcelid','logerror','month']],
                           left_on='ParcelId', right_on='parcelid')
    oct_error = abs(trainresults[trainresults['month'] == 10]['201610'] 
                    - trainresults[trainresults['month'] == 10]['logerror']).mean()
    nov_error = abs(trainresults[trainresults['month'] == 11]['201611'] 
                    - trainresults[trainresults['month'] == 11]['logerror']).mean()
    dec_error = abs(trainresults[trainresults['month'] == 12]['201612'] 
                    - trainresults[trainresults['month'] == 12]['logerror']).mean()
    overall_mae = (oct_error*(trainresults['month'] == 10).sum() + nov_error*(trainresults['month'] == 11).sum() 
                        + dec_error*(trainresults['month'] == 12).sum()) / (trainresults['month'].isin([10,11,12])).sum()
    return (oct_error, nov_error, dec_error, overall_mae)

## Sample Weights

In [49]:
p75 = np.percentile(data['logerror'],75)
p25 = np.percentile(data['logerror'],25)
extremelogerrors = ((data['logerror'] >= p75) | (data['logerror'] <= p25))*1
data['wts_oct'] = extremelogerrors + np.where(data['month'] == 10, 1.5, 1)
data['wts_nov'] = np.where(data['month'] == 11, 1.5, 1)
data['wts_dec'] = np.where(data['month'] == 12, 1.5, 1)

## Train/Test Set Split

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
traindata, testdata = train_test_split(data, test_size = 0.2, random_state=9)
traindata = traindata.reset_index(drop=True)
testdata = testdata.reset_index(drop=True)

# Random Forests

In [52]:
X_train = preprocess(data)
Y_train = data['logerror']

In [53]:
from sklearn.ensemble import RandomForestRegressor

### Grid Search

In [30]:
# param_grid = dict(max_depth=[5,10,15,20,25])

# oct_gs = GridSearchCV(RandomForestRegressor(random_state=42, n_estimators = 200, max_features = 9),
#                       param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# nov_gs = GridSearchCV(RandomForestRegressor(random_state=42, n_estimators = 200, max_features = 9),
#                       param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# dec_gs = GridSearchCV(RandomForestRegressor(random_state=42, n_estimators = 200, max_features = 9),
#                       param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# print("Training")
# print("training oct model...")
# oct_gs.fit(X_train,Y_train, sample_weight=data['wts_oct'])
# print("training nov model...")
# nov_gs.fit(X_train,Y_train, sample_weight=data['wts_nov'])
# print("training dec model...")
# dec_gs.fit(X_train,Y_train, sample_weight=data['wts_nov'])

# submission_df = generate_submissions(oct_gs.best_estimator_, 
#                                      nov_gs.best_estimator_, 
#                                      dec_gs.best_estimator_, name ="RF_submission_mean")
# mean_absolute_errors(submission_df, data)

In [54]:
oct_regr_rf = RandomForestRegressor(n_estimators = 100, max_features = 9, random_state=42, max_depth=35, criterion='mse')
nov_regr_rf = RandomForestRegressor(n_estimators = 100, max_features = 9, random_state=42, max_depth=35, criterion='mse')
dec_regr_rf = RandomForestRegressor(n_estimators = 100, max_features = 9, random_state=42, max_depth=35, criterion='mse')

oct_regr_rf.fit(X_train,Y_train, sample_weight=data['wts_oct'])
nov_regr_rf.fit(X_train,Y_train, sample_weight=data['wts_nov'])
dec_regr_rf.fit(X_train,Y_train, sample_weight=data['wts_dec'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=35,
           max_features=9, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [55]:
submission_df = generate_submissions(oct_regr_rf, nov_regr_rf, dec_regr_rf)

In [56]:
submission_df.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.0712,0.1548,0.0899,0,0,0
1,10759547,-0.0444,0.0120,-0.0269,0,0,0
2,10843547,0.0310,-0.0269,0.0686,0,0,0
3,10859147,0.2890,0.1305,0.1478,0,0,0
4,10879947,0.2394,0.1818,0.1399,0,0,0


In [57]:
mean_absolute_errors(submission_df, data)

(0.052186437613020095,
 0.05355925520262865,
 0.06255405405405404,
 0.054590564270662709)

In [58]:
submission_df[['201610','201611','201612']].std()

201610    0.049015
201611    0.050554
201612    0.049034
dtype: float64

## Model Averaging: XGB + RF

In [59]:
XGB = pd.read_csv("XGB_3000.csv")

In [67]:
XGB[['201610','201611','201612']].describe()

,201610,201611,201612
count,2.985217e+06,2.985217e+06,2.985217e+06
mean,1.715665e-02,1.743313e-02,1.726862e-02
std,6.656845e-02,6.871725e-02,6.613157e-02
min,-2.266000e+00,-2.233000e+00,-2.214000e+00
25%,4.449000e-03,4.359000e-03,4.401000e-03
50%,1.279000e-02,1.277000e-02,1.278000e-02
75%,2.281000e-02,2.284000e-02,2.289000e-02
max,3.834000e+00,3.803000e+00,3.868000e+00


In [60]:
mean_absolute_errors(XGB, data)

(0.05998706659714675,
 0.05918412810186208,
 0.07170160296434737,
 0.06220029687696086)

In [80]:
blend = (((1/2)*submission_df + (1/2)*XGB))
blend['ParcelId'] = blend['ParcelId'].astype(int)
assert all(blend.ParcelId.unique() == submission_df.ParcelId.unique())
mean_absolute_errors(blend, data)

(0.05430681879987949,
 0.054777713722343975,
 0.06490518643041981,
 0.05656512077107239)

In [64]:
blend[['201610','201611','201612']].describe()

,201610,201611,201612
count,2.985217e+06,2.985217e+06,2.985217e+06
mean,1.665456e-02,1.862669e-02,2.154185e-02
std,4.856857e-02,5.005677e-02,4.817773e-02
min,-2.102850e+00,-2.118050e+00,-2.022400e+00
25%,1.310000e-03,3.860000e-03,6.726600e-03
50%,1.398300e-02,1.600500e-02,1.910500e-02
75%,2.684500e-02,2.834000e-02,3.210000e-02
max,3.429000e+00,3.340500e+00,3.319050e+00


In [68]:
blend.to_csv("XGB_RF_overfit.gz", index=False, float_format='%.4g', compression='gzip')

In [70]:
RF = pd.read_csv("RF_submission_mean.csv")
mean_absolute_errors(RF, data)

(0.06266067912397041,
 0.0618791894852136,
 0.07436026451983876,
 0.064875450714118515)

In [84]:
RF.shape

(2985217, 7)

In [95]:
blend2.ParcelId.nunique()

2985217

In [106]:
blend2 = submission_df[['ParcelId']].copy()

In [107]:
cols = ['201610','201611','201612','201710','201711','201712']
blend2 = pd.concat([blend2,(1/10)*submission_df[cols] + (1/10)*XGB[cols] + (8/10)*RF[cols]], axis=1)
blend2['ParcelId'] = blend2['ParcelId'].astype(int)
assert all(blend2.ParcelId.unique() == submission_df.ParcelId.unique())
mean_absolute_errors(blend2, data)

(0.06053106576640541,
 0.05996690064151146,
 0.07188351468688907,
 0.062721623382264072)

In [108]:
blend2.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.125112,0.174630,0.184297,0.0,0.0,0.0
1,10759547,-0.013062,-0.002501,-0.008912,0.0,0.0,0.0
2,10843547,0.254780,0.206110,0.172340,0.0,0.0,0.0
3,10859147,0.221960,0.190970,0.203410,0.0,0.0,0.0
4,10879947,0.039744,0.012240,0.071535,0.0,0.0,0.0


In [109]:
blend2.to_csv("XGB_RF_RF_overfit.gz", index=False, float_format='%.4g', compression='gzip')